In [ ]:
!pip3 install -q arize langchain

# 1. Set Arize credentials and define the callback handler 

In [ ]:
from langchain.callbacks.arize_callback import ArizeCallbackHandler

SPACE_KEY = "YOUR_SPACE_KEY"
API_KEY =  "YOUR_API_KEY"

if SPACE_KEY == "YOUR_SPACE_KEY" or API_KEY == "YOUR_API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")

# Define callback handler for Arize
arize_chat_callback = ArizeCallbackHandler(
model_id="anyscale-langchain-demo",
model_version="1.0",
SPACE_KEY=SPACE_KEY,
API_KEY=API_KEY
)

# 2. Use LangChain to define prompt messages, get models, and directly call Arize callback

In [ ]:
from langchain.schema import SystemMessage, HumanMessage
from langchain.chat_models import ChatAnyscale

ANYSCALE_ENDPOINT_TOKEN = "YOUR_ENDPOINT_TOKEN"
messages = [
    SystemMessage(
        content="You are a helpful AI that shares everything you know."
    ),
    HumanMessage(
        content="How to evaluate the value of a NFL team"
    ),
]

chats = {
    model: ChatAnyscale(anyscale_api_key=ANYSCALE_ENDPOINT_TOKEN,
                        model_name=model, 
                        temperature=1.0)
    for model in ChatAnyscale.get_available_models(anyscale_api_key=ANYSCALE_ENDPOINT_TOKEN)
}

for model, chat in chats.items():
    response = chat.predict_messages(messages, callbacks=[arize_chat_callback])
    print(model, "\n", response.content)

# 3. Use callback manager for Arize integration

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import StdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

manager = CallbackManager([StdOutCallbackHandler(), arize_chat_callback])

In [ ]:
template = """You are a playwrighter. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=chat, prompt=prompt_template, callback_manager=manager)


overall_chain = SimpleSequentialChain(
   chains=[synopsis_chain], callback_manager=manager
)


test_prompts = [
   {
       "input": "documentary about pandas who are about be extinct because of global warming"
   },
   {"input": "once upon a time in hollywood"},
   {"input": "the best model observability tooling"},
   {"input": "childrens play about a frog living in an alpine lake just discovered by humans"},
   {"input": "utopian society being disrupted by new AI"},
]
overall_chain.apply(test_prompts)